In [28]:
import numpy as np
from utils.utils import normalizeRows, softmax
from word2vec import getNegativeSamples

In [29]:
import word2vec

def sigmoid(x):
    """
    Compute the sigmoid function for the input here.
    Arguments:
    x -- A scalar or numpy array.
    Return:
    s -- sigmoid(x)
    """

    ### YOUR CODE HERE (~1 Line)
    s = 1 / (1 + np.exp(-x))
    ### END YOUR CODE

    return s

word2vec.sigmoid = sigmoid

In [30]:
from word2vec import sigmoid
sigmoid(np.zeros(2))

array([0.5, 0.5])

In [31]:
def naiveSoftmaxLossAndGradient(
    centerWordVec,
    outsideWordIdx,
    outsideVectors,
    dataset
):
    """ Naive Softmax loss & gradient function for word2vec models
    Implement the naive softmax loss and gradients between a center word's 
    embedding and an outside word's embedding. This will be the building block
    for our word2vec models.
    Arguments:
    centerWordVec -- numpy ndarray, center word's embedding
                    (v_c in the pdf handout)
    outsideWordIdx -- integer, the index of the outside word
                    (o of u_o in the pdf handout)
    outsideVectors -- outside vectors (rows of matrix) for all words in vocab
                      (U in the pdf handout)
    dataset -- needed for negative sampling, unused here.
    Return:
    loss -- naive softmax loss
    gradCenterVec -- the gradient with respect to the center word vector
                     (dJ / dv_c in the pdf handout)
    gradOutsideVecs -- the gradient with respect to all the outside word vectors
                    (dJ / dU)
    """

    ### YOUR CODE HERE

    ### Please use the provided softmax function (imported earlier in this file)
    ### This numerically stable implementation helps you avoid issues pertaining
    ### to integer overflow.
    y_hat = softmax(np.dot(centerWordVec, outsideVectors.T))
    delta = y_hat.copy()
    delta[outsideWordIdx] -= 1

    loss = -np.log(y_hat)[outsideWordIdx]
    gradCenterVec = np.dot(delta, outsideVectors)
    gradOutsideVecs = np.dot(delta[:, np.newaxis], centerWordVec[np.newaxis, :])

    ### END YOUR CODE

    return loss, gradCenterVec, gradOutsideVecs

word2vec.naiveSoftmaxLossAndGradient = naiveSoftmaxLossAndGradient

In [32]:
def negSamplingLossAndGradient(
    centerWordVec,
    outsideWordIdx,
    outsideVectors,
    dataset,
    K=10
):
    """ Negative sampling loss function for word2vec models
    Implement the negative sampling loss and gradients for a centerWordVec
    and a outsideWordIdx word vector as a building block for word2vec
    models. K is the number of negative samples to take.
    Note: The same word may be negatively sampled multiple times. For
    example if an outside word is sampled twice, you shall have to
    double count the gradient with respect to this word. Thrice if
    it was sampled three times, and so forth.
    Arguments/Return Specifications: same as naiveSoftmaxLossAndGradient
    """

    # Negative sampling of words is done for you. Do not modify this if you
    # wish to match the autograder and receive points!
    negSampleWordIndices = getNegativeSamples(outsideWordIdx, dataset, K)
    indices = [outsideWordIdx] + negSampleWordIndices

    ### YOUR CODE HERE

    ### Please use your implementation of sigmoid in here.
    gradOutsideVecs = np.zeros(outsideVectors.shape)
    gradCenterVec = np.zeros(centerWordVec.shape)
    loss = 0.0
    # allVecs = outsideVectors[indices]
    # allScores = np.dot(allVecs, centerWordVec)

    # loss = - np.log(sigmoid(allScores[0])) - np.sum(np.log(sigmoid(-allScores[1:])))
    # # maybe the equation below is wrong
    # gradCenterVec = (allScores[0] - 1) * allVecs[0] + (1 - sigmoid(-allScores[1:])).dot(allVecs[1:])

    # gradOutsideVecs = np.zeros_like(outsideVectors)
    # gradOutsideVecs[outsideWordIdx] -= (1 - sigmoid(allScores[0])) * centerWordVec
    # for negSampleWordIdx, i in zip(negSampleWordIndices, range(1, len(indices))):
    #     gradOutsideVecs[negSampleWordIdx] += (
    #         1 - sigmoid(-allScores[i])) * allVecs[i]
    z = sigmoid(np.dot(outsideVectors[outsideWordIdx], centerWordVec))
    loss -= np.log(z)

    gradOutsideVecs[outsideWordIdx] += centerWordVec * (z - 1.0)
    gradCenterVec += outsideVectors[outsideWordIdx] * (z - 1.0)

    # Implementation with for loop
    #for k in range(K):
    #    samp = indices[k+1]
    #    z = sigmoid(np.dot(-outsideVectors[samp], centerWordVec))
    #    loss -= np.log(z)
    #    gradOutsideVecs[samp] -=  centerWordVec * (z - 1.0)
    #    gradCenterVec -= outsideVectors[samp] * (z - 1.0)
    # Vectorized implementation
    u_k = outsideVectors[negSampleWordIndices]
    z = sigmoid(-np.dot(u_k,centerWordVec))
    loss += np.sum(- np.log(z))
    gradCenterVec += np.dot((z-1),u_k)*(-1)
    gradOutsideVecs[negSampleWordIndices] += np.outer((z-1),centerWordVec)*(-1)
    
    ### END YOUR CODE

    return loss, gradCenterVec, gradOutsideVecs

word2vec.negSamplingLossAndGradient = negSamplingLossAndGradient

In [33]:
def skipgram(currentCenterWord, windowSize, outsideWords, word2Ind,
             centerWordVectors, outsideVectors, dataset,
             word2vecLossAndGradient=naiveSoftmaxLossAndGradient):
    """ Skip-gram model in word2vec
    Implement the skip-gram model in this function.
    Arguments:
    currentCenterWord -- a string of the current center word
    windowSize -- integer, context window size
    outsideWords -- list of no more than 2*windowSize strings, the outside words
    word2Ind -- a dictionary that maps words to their indices in
              the word vector list
    centerWordVectors -- center word vectors (as rows) for all words in vocab
                        (V in pdf handout)
    outsideVectors -- outside word vectors (as rows) for all words in vocab
                    (U in pdf handout)
    word2vecLossAndGradient -- the loss and gradient function for
                               a prediction vector given the outsideWordIdx
                               word vectors, could be one of the two
                               loss functions you implemented above.
    Return:
    loss -- the loss function value for the skip-gram model
            (J in the pdf handout)
    gradCenterVecs -- the gradient with respect to the center word vectors
            (dJ / dV in the pdf handout)
    gradOutsideVectors -- the gradient with respect to the outside word vectors
                        (dJ / dU in the pdf handout)
    """

    loss = 0.0
    gradCenterVecs = np.zeros(centerWordVectors.shape)
    gradOutsideVectors = np.zeros(outsideVectors.shape)

    ### YOUR CODE HERE
    currentCenterWordIdx = word2Ind[currentCenterWord]
    centerWordVec = centerWordVectors[currentCenterWordIdx]

    for outsideWord in outsideWords:
        outsideWordIdx = word2Ind[outsideWord]
        (l, gradCenter, gradOutside) = word2vecLossAndGradient(
            centerWordVec, outsideWordIdx, outsideVectors, dataset)
        loss += l
        gradCenterVecs[currentCenterWordIdx] += gradCenter
        gradOutsideVectors += gradOutside

    ### END YOUR CODE

    return loss, gradCenterVecs, gradOutsideVectors

word2vec.skipgram = skipgram


In [34]:
#from sgd import sgd

In [35]:
def sgd(f, x0, step, iterations, postprocessing=None, useSaved=False,
        PRINT_EVERY=10):
    """ Stochastic Gradient Descent

    Implement the stochastic gradient descent method in this function.

    Arguments:
    f -- the function to optimize, it should take a single
         argument and yield two outputs, a loss and the gradient
         with respect to the arguments
    x0 -- the initial point to start SGD from
    step -- the step size for SGD
    iterations -- total iterations to run SGD for
    postprocessing -- postprocessing function for the parameters
                      if necessary. In the case of word2vec we will need to
                      normalize the word vectors to have unit length.
    PRINT_EVERY -- specifies how many iterations to output loss

    Return:
    x -- the parameter value after SGD finishes
    """

    # Anneal learning rate every several iterations
    ANNEAL_EVERY = 20000

    if useSaved:
        start_iter, oldx, state = load_saved_params()
        if start_iter > 0:
            x0 = oldx
            step *= 0.5 ** (start_iter / ANNEAL_EVERY)

        if state:
            random.setstate(state)
    else:
        start_iter = 0

    x = x0

    if not postprocessing:
        postprocessing = lambda x: x

    exploss = None

    for iter in range(start_iter + 1, iterations + 1):
        # You might want to print the progress every few iterations.

        loss = None
        ### YOUR CODE HERE (~2 lines)
        loss, gradients = f(x)
        x -= step * gradients
        ### END YOUR CODE

        x = postprocessing(x)
        if iter % PRINT_EVERY == 0:
            if not exploss:
                exploss = loss
            else:
                exploss = .95 * exploss + .05 * loss
            print("iter %d: %f" % (iter, exploss))

        if iter % SAVE_PARAMS_EVERY == 0 and useSaved:
            save_params(iter, x)

        if iter % ANNEAL_EVERY == 0:
            step *= 0.5

    return x

In [36]:
from word2vec import test_word2vec
test_word2vec()


==== Gradient check for skip-gram with naiveSoftmaxLossAndGradient ====
Gradient check passed!. Read the docstring of the `gradcheck_naive` method in utils.gradcheck.py to understand what the gradient check does.
======Skip-Gram with naiveSoftmaxLossAndGradient Test Cases======
The first test passed!
The second test passed!
The third test passed!
All 3 tests passed!
==== Gradient check for skip-gram with negSamplingLossAndGradient ====
Gradient check failed for negSamplingLossAndGradient Gradient.
First gradient error found at index (5, 0) in the vector of gradients
Your gradient: -1.258186 	 Numerical gradient: -3.398960
======Skip-Gram with negSamplingLossAndGradient======


AssertionError: Your gradOutsideVectors do not match expected gradOutsideVectors.